In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
3.1


In [3]:
from ema_workbench import (Model, 
                           CategoricalParameter,
                           ScalarOutcome, 
                           IntegerParameter, 
                           RealParameter,
                           ema_logging
                           )
from dike_model_function import DikeNetwork  # @UnresolvedImport


'''def sum_over(*args):
    return sum(args)'''

'def sum_over(*args):\n    return sum(args)'

In [ ]:
from ema_workbench import (MultiprocessingEvaluator, 
                           SequentialEvaluator, 
                           Model, 
                           Policy, 
                           Scenario,
                           perform_experiments,
                           save_results,
                           load_results)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [5]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [6]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = copy.deepcopy(dike_model.uncertainties)



CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [7]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [8]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A3 Expected Annual Damage', variable_name=('A.3_Expected Annual Damage 0', 'A.3_Expected Annual Damage 1', 'A.3_Expected Annual Damage 2'), function=<function sum_over at 0x1540add00>)
ScalarOutcome('A3 Aggr Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths 0', 'A.3_Expected Number of Deaths 1', 'A.3_Expected Number of Deaths 2'), function=<function sum_over at 0x1540add00>)
ScalarOutcome('A3 Dike Investment Costs', variable_name=('A.1_Dike Investment Costs 0', 'A.2_Dike Investment Costs 0', 'A.3_Dike Investment Costs 0', 'A.4_Dike Investment Costs 0', 'A.5_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.2_Dike Investment Costs 1', 'A.3_Dike Investment Costs 1', 'A.4_Dike Investment Costs 1', 'A.5_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2', 'A.2_Dike Investment Costs 2', 'A.3_Dike Investment Costs 2', 'A.4_Dike Investment Costs 2', 'A.5_Dike Investment Costs 2'), function=<function sum_over at 0x1540add00>)
ScalarOutcome('

In [9]:
# with SequentialEvaluator(dike_model) as evaluator:
#     results = evaluator.perform_experiments(scenarios=50, policies=4)

with MultiprocessingEvaluator(dike_model, n_processes=7) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=20, policies=1)

[MainProcess/INFO] pool started with 7 workers
[MainProcess/INFO] performing 20 scenarios * 1 policies * 1 model(s) = 20 experiments
100%|██████████████████████████████████████████| 20/20 [00:02<00:00,  6.85it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [10]:
save_results([experiments, outcomes], '2000 scenarios 1 policy PF3.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/2000 scenarios 1 policy PF3.tar.gz


In [12]:
experiments, outcomes = load_results('2000 scenarios 1 policy PF3.tar.gz')

outcomes

{'A3 Expected Annual Damage': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'A3 Aggr Expected Number of Deaths': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'A3 Dike Investment Costs': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Room for River Investment Costs': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Evacuation Costs': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

## Need better documentation for the policies! (See cell below)

## If anyone finds an explanation on what the dictionary keys mean, please insert here! 

In [ ]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{'0_RfR 0':1,
                                  '0_RfR 1':1,
                                  '0_RfR 2':1,
                                  'A.1_DikeIncrease 0':5}),
           Policy('policy 2', **{'4_RfR 0':1,
                                  '4_RfR 1':1,
                                  '4_RfR 2':1,
                                  'A.5_DikeIncrease 0':5}),
           Policy('policy 3', **{'1_RfR 0':1,
                                  '2_RfR 1':1,
                                  '3_RfR 2':1,
                                  'A.3_DikeIncrease 0':5})]

In [ ]:
from ema_workbench import load_results

results = load_results('2000 scenarios 1 policy PF3.tar.gz')

#pass the policies list to EMA workbench experiment runs
n_scenarios = 100

with MultiprocessingEvaluator(dike_model, n_processes=7) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,policies)

[MainProcess/INFO] results loaded successfully from /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/2000 scenarios 1 policy PF3.tar.gz


KeyboardInterrupt: 